<a href="https://colab.research.google.com/github/aaalexlit/wandb_notebooks/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb tenacity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [2]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)
import wandb
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [3]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
··········
OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [4]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [5]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [6]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [7]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))

generate_and_print(system_prompt, user_prompt)

"Hi there! I've been using W&B for my machine learning experiments, and I'm having trouble with viewing my         
experiment's figures in the W&B dashboard. Can you help me troubleshoot this issue?"

"How can I visualize the training progress of my machine learning model using Weights & Biases?"

"Can you help me understand how to track and visualize my machine learning model's training progress using Weights 
& Biases?"

Why am I unable to log in to my W&B account?

"What are the steps to integrate Weights & Biases with my machine learning project?"

# Few Shot

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [21]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

--2023-08-31 15:29:28--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40595 (40K) [text/plain]
Saving to: ‘examples.txt’

examples.txt        100%[===================>]  39.64K  --.-KB/s    in 0.005s  

2023-08-31 15:29:28 (7.84 MB/s) - ‘examples.txt’ saved [40595/40595]



In [22]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: " i have my log from training in a txt file. Can i import that data to wandb so i can visualize the    
data? I have accuracy and loss logged every 50 steps on training data, and every 5000 steps on validation data. How
can I plot those two different accuracies and their losses?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [23]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: @Corey_S pls  
note i can log a wandb.image of a tensor of floats but not of a tensor of ints Right now I am grabbing run         
summaries and artifacts by tags how might I get runs by the sweep names instead can i use wandb with slurm? Let's  
start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [24]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

  1 "I'm currently running my experiments using PyTorch. Can I integrate Weights and Biases with PyTorch for better
    run management and visualization of my models?"                                                                
  2 "I'm constantly running multiple experiments, how can I use Weights and Biases to compare different runs?"     
  3 "Can I track hyperparameters of my machine learning model using Weights and Biases? If so, could you guide me  
    on how to do it?"                                                                                              
  4 "Is there a way to log and visualize my model's performance over time in Weights and Biases?"                  
  5 "I've been coding in Jupyter notebooks and I heard W&B has a Jupyter extension. How can I make use of it?"     
  6 "Can I integrate Weights and Biases with my TensorFlow code? What is the process for that?"                    
  7 "How to set up Weights and Biases for tracking a long running job/client?"                                     
  8 "What's the process of integrating Weights and Biases with Keras?"                                             
  9 "Is there a way to log custom metrics in Weights and Biases?"                                                  
 10 "How do I save, load and version my datasets using Weights and Biases?"                                        
 11 "How to use Weights and Biases to monitor a PyTorch Lightning training?"                                       
 12 "Does Weights and Biases support ROC Curve visualization? If so, how can I log it?"                            
 13 "Can I log 3D objects for visualization in Weights and Biases?"                                                
 14 "Is there a way to notify me through email or other platforms when a run finishes or encounters a problem?"    
 15 "How to retrieve artifact metadata from Weights and Biases after a run?"                                       
 16 "Can Weights and Biases be used for distributed training? If yes, how would one set that up?"                  
 17 "What are the steps to log the architecture of my machine learning model in Weights and Biases?"               
 18 "How are API keys managed in Weights and Biases, and can I generate multiple ones?"                            
 19 "Can I share my Weights and Biases project dashboard with collaborators who don't have an account?"            
 20 "How do I log my scikit-learn model's metrics to Weights and Biases?"

  1 "Is it possible to log histograms in W&B using PyTorch?"                                                       
  2 "What's the way to visualize the confusion matrix in W&B?"                                                     
  3 "How could I save and restore models in W&B?"                                                                  
  4 "Why am I not able to sync my offline run on W&B?"                                                             
  5 "Can W&B be integrated with TensorFlow?"                                                                       
  6 "How do I control the frequency of my logging in W&B?"                                                         
  7 "Can I use W&B to log hyperparameters for my machine learning model?"                                          
  8 "How do I start a new W&B run for my second experiment?"                                                       
  9 "Is it possible to update a previous run datas after the run has ended in W&B?"                                
 10 "Is there a way to change the visibility of a run in W&B?"                                                     
 11 "How to prevent W&B from recording my training scripts?"                                                       
 12 "Is W&B able to plot a correlation matrix?"                                                                    
 13 "Can I store large files as artifacts in W&B?"                                                                 
 14 "Why is my W&B dashboard not showing any data?"                                                                
 15 "How to share a report of my run with my team in W&B?"                                                         
 16 "Can I compare different model architectures using W&B?"                                                       
 17 "Can I track my model's performance across multiple runs in W&B?"                                              
 18 "Is there a way to resume a run in W&B after an unintended interruption?"                                      
 19 "Why does W&B fail to authenticate when I try and log in?"                                                     
 20 "Does W&B provide any feature for autoML workflows?"

  1 "Hey there, I'm quite new to W&B. Can you guide me on how to sync my offline runs once I get back online?"     
  2 "I'm trying to log a wandb.image with a tensor of ints, but it doesn't seem to work. Can you please help me    
    figure this out?"                                                                                              
  3 "I was wondering if it is possible to retrieve the details of the projects to which I have access using the    
    API?"                                                                                                          
  4 "I'm working on a project where I want to log multiple charts on the same graph, is it possible using W&B?"    
  5 "I'm using a Docker container and want to perform regular sweeps. Can you help me understand how to do this    
    using W&B?"                                                                                                    
  6 "Is there a way to set up notifications for when a run fails or finishes early in W&B?"                        
  7 "Hi, can you guide me on how to use W&B with Kaggle notebooks?"                                                
  8 "I need to manage the tags of the runs for my project. Can I add or remove them programmatically using the W&B 
    API?"                                                                                                          
  9 "I would like to know if there's a way to compare two or more runs using W&B."                                 
 10 "Is there an option to clone a run in W&B? If yes, can you guide me on how to do it?"

  1 "Hi, I'm trying to compare two different models in the same project using W&B, how should I go for this?"      
  2 "Can I use W&B to log the hyperparameters of my deep learning model, and if so, how?"                          
  3 "I'm working with Jupyter Notebooks, is there any way to integrate W&B there?"                                 
  4 "Is it possible to pause and resume logging in the middle of training using W&B?"                              
  5 "Hello, how can I visually compare different runs with W&B?"                                                   
  6 "I want to log my custom metrics in W&B, how do I do this?"                                                    
  7 "Can W&B support logging and visualizing embeddings?"                                                          
  8 "Is there a way to link artifacts in W&B to specific runs or experiments?"                                     
  9 "How can I log different types of plots like ROC curve or confusion matrix using W&B?"                         
 10 "I'm using PyTorch for my project. How can I integrate W&B into my workflow?"

  1 "How can I collaborate with my team using Weights & Biases? Is there a way to share projects or experiments?"  
  2 "I'm a beginner. Can you guide me on how to start a new project in Weights & Biases?"                          
  3 "Can Weights & Biases integrate with PyTorch for logging metrics and visualizations?"                          
  4 "Is it possible to track hyperparameters using Weights & Biases, and if so, how do I go about this?"           
  5 "I'm trying to log my model's performance for each epoch in Weights & Biases. Could you provide some help      
    regarding this?"                                                                                               
  6 "I am working on a machine learning model that takes a long time to train. How can I use Weights & Biases to   
    resume interrupted runs?"                                                                                      
  7 "Where can I view a list of all my experiment runs and their corresponding details in Weights & Biases?"       
  8 "How do I log my model's gradients and weights in Weights & Biases during training?"                           
  9 "How can I use Weights & Biases to compare different models and their performances?"                           
 10 "What's the best way to store and retrieve data artifacts using Weights & Biases?"                             
 11 "Is there a way to integrate Weights & Biases with my TensorFlow workflows?"                                   
 12 "How can I customize the charts on the Weights & Biases dashboard to better analyze my data?"                  
 13 "Can Weights & Biases help me track my model's inference time, specifically for deployment?"                   
 14 "I need to manage multiple machine learning projects simultaneously. Does Weights & Biases offer any project   
    management tools?"                                                                                             
 15 "Can I schedule and automate my model training tasks through Weights & Biases?"                                
 16 "How do I use Weights & Biases to keep track of my code versions and changes?"                                 
 17 "Can I use Weights & Biases for automated hyperparameter tuning?"                                              
 18 "How do I upload my custom datasets to Weights & Biases?"                                                      
 19 "I am using Keras for my projects. How do I integrate it with Weights & Biases?"                               
 20 "Does Weights & Biases support real-time collaboration? How can my team and I work together on a project?"

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [11]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

Cloning into 'edu'...
remote: Enumerating objects: 2493, done.
remote: Counting objects: 100% (1036/1036), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 2493 (delta 722), reused 858 (delta 641), pack-reused 1457
Receiving objects: 100% (2493/2493), 22.61 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (1418/1418), done.


In [25]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [26]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[956, 365, 1206, 2596, 2093, 2529, 803, 537, 4179, 2940, 1644]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [19]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [27]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [12]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: -page.md#search-for-runs) of the ]8;id=514735;../pages/project-page.md\Project Page]8;;\, 
click the table icon in the upper-right.  This will expand the sidebar into the full ]8;id=918419;runs-table.md\runs table]8;;\.                   

Hover over a run in the table to see a checkbox on the left or look in the header row for a checkbox that will     
allow you to select all runs.                                                                                      

Click the checkbox to enable bulk actions. Select the runs to which you'd like to apply your tag(s).               

Click the Tag button above the rows of runs.                                                                       

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  


This method is best suited to applying a tag or tags to a single run by hand.                                      

In the left sidebar of the ]8;id=292450;../pages/run-page.md\Run Page]8;;\, click the top ]8;id=450036;../pages/run-page.md#overview-tab\Overview tab]8;;\.                                                   

Next to "Tags" is a gray ➕ button. Click on that plus to add a tag.                                               

Type a tag you'd like to add and click "Add" below the text box to add a new tag.                                  



                                                How to remove tags                                                 

Tags can also be removed from runs via the UI.                                                                     


<Tabs defaultValue="projectpage" values={[ {label: 'Project Page', value: 'projectpage'}, {label: 'Run Page',      
value: 'runpage'}, ]}>                                                                                             

This method is best suited to removing tags from a large numbers of runs.                                          

In the ]8;id=124066;../pages/project-page.md#search-for-runs\runs sidebar]8;;\ of the ]8;id=778728;../pages/project-page.md\Project Page]8;;\,  click the table icon in the upper-right.  This will expand the sidebar   
into the full ]8;id=763395;runs-table.md\runs table]8;;\.                                                                                          

Hover over a run in the table to see a checkbox on the left or look in the header row for a checkbox that will     
allow you to select all runs.                                                                                      

Click either checkbox to enable bulk actions. Select the runs to from which you'd like to remove your tag(s).      

Click the Tag button above the rows of runs.                                                                       

Click the checkbox next to a tag to remove it from the run.                                                        


In the left sidebar of the ]8;id=328683;../pages/run-page.md\Run Page,]8;;\ click the top [Overview tab](../pages/run-page.md Let's start!

Let's generate 3 possible questions:

In [15]:
generate_and_print(system_prompt, generation_prompt, n=3)

How do I apply tags to multiple runs at once in W&B?

Question: How can I remove tags from multiple runs?                                                                

Answer: To remove tags from multiple runs, you can follow these steps:                                             

 1 Go to the Project Page.                                                                                         
 2 Click on the table icon in the upper-right corner of the runs sidebar.                                          
 3 Hover over a run in the table to see a checkbox on the left or look for a checkbox in the header row to select  
   all runs.                                                                                                       
 4 Click the checkbox to enable bulk actions.                                                                      
 5 Select the runs from which you want to remove the tag(s).                                                       
 6 Click the Tag button above the rows of runs.                                                                    
 7 Click the checkbox next to the tag you want to remove from the runs.                                            
 8 The tag will be removed from the selected runs.

How can I remove tags from multiple runs on the Project Page in W&B?

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [8]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [13]:
# read system_template.txt file into an f-string
with open("/content/edu/llm-apps-course/notebooks/system_template.txt", "r") as file:
    system_prompt = file.read()

In [14]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [15]:
# read prompt_template.txt file into an f-string
with open("/content/edu/llm-apps-course/notebooks/prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [16]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [28]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [29]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
how can i login with a different wandb user? どんなことができますか？ what is the capital of Ireland?              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
b.init(tags=["tag1", "tag2"])` .                                                                                   

You can also update the tags of a run during training (e.g. if a particular metrics crosses a pre-defined          
threshold):                                                                                                        

                                                                                                                   
 run = wandb.init(entity="entity", project="capsules", tags=["debug"])                                             
                                                                                                                   
 ...                                                                                                               
                                                                                                                   
 if current_loss < threshold:                                                                                      
     run.tags = run.tags + ("release_candidate",)                                                                  
                                                                                                                   

There are also several ways to add tags after runs have been logged to Weights & Biases.                           


<Tabs defaultValue="publicapi" values={[ {label: 'Using the Public API', value: 'publicapi'}, {label: 'Project     
Page', value: 'projectpage'}, {label: 'Run Page', value: 'runpage'}, ]}>                                           

After a run is created, you can update tags using ]8;id=464917;../../../guides/track/public-api-guide.md\our public API]8;;\ like so:                                          

                                                                                                                   
 run = wandb.Api().run("{entity}/{project}/{run-id}"})                                                             
 run.tags.append("tag1")  # you can choose tags based on run data here                                             
 run.update()                                                                                                      
                                                                                                                   

You can read more about how to use the Public API in the ]8;id=220914;../../../ref/README.md\reference documentation]8;;\ or ]8;id=33868;../../../guides/track/public-api-guide.md\guide]8;;\.                         


This method is best suited to tagging large numbers of runs with the same tag or tags.                             

In the ]8;id=39826;../pages/project-page.md#search-for-runs\runs sidebar]8;;\ of the ]8;id=410138;../pages/project-page.md\Project Page]8;;\,  click the table icon in the upper-right.  This will expand the sidebar   
into the full ]8;id=236874;runs-table.md\runs table]8;;\.                                                                                          

Hover over a run in the table to see a checkbox on the left or look in the header ro

In [30]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [31]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None

    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [32]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

("\nThe user is trying to manage tags associated with their runs in W&B. They've created a number of runs as part of an experiment and have tagged each run with relevant information. However, they've decided to do some organization and cleanup of these tags and are not sure how to proceed.\n",
 '\nHow can I bulk remove tags from numerous runs all at once on the Project Page?\n',
 '\nYou can easily remove tags from multiple runs on the Project Page. Here\'s how to do it:\n\n1. Navigate to the [runs sidebar](../pages/project-page.md#search-for-runs) on the Project Page and  click the table icon in the upper-right. This expands the sidebar into the full [runs table](runs-table.md).\n2. Then, hover over a run in the table to see a checkbox on the left, or look in the header row for a checkbox that will allow you to select all runs.\n3. Click the checkbox to enable bulk actions and select the runs from which you want to remove your tag(s).\n4. Unfortunately, the document is cut off at this 

In [33]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

<Artifact generated_examples>

In [34]:
wandb.finish()

usage/completion_tokens,▁▇▆▇▄▇▇▅█▆██▅▆
usage/elapsed_time,▁▇▄▇▄▇▆▆▆▇█▇▅▇
usage/prompt_tokens,▁▂▅▆▅▆▆▆▆▅█▅▅▆
usage/total_tokens,▁▅▆▆▄▇▆▅▇▆█▇▅▆
usage/completion_tokens,1580
usage/elapsed_time,55.76169
usage/prompt_tokens,941
usage/total_tokens,2521
